# Required Lift and optimum L/D

<img src="img/unstallable00.png" alt="Pic here" style="width:700px">

## Geometry and total mass

The lift required will be equal to the weight of the aircraft, hence, let's parameterized the weight as a function of geometric parameter. 

First off, we have the following base weights:

<strong>Component &nbsp; &nbsp;Estimated Weight (grams/each)</strong><br>ESC (2): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 9<br>BEC (1): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 16<br>Raspberry Pi 3&nbsp;(1): 47<br>Flip32&nbsp;(1): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 14<br>Battery&nbsp;(1): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 36<br>Propeller&nbsp;(1): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;3<br>GPS&nbsp;(1): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 32<br>Motor&nbsp;(2): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15<br>Servos (2-4): &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;9<br>Wires, connectors, etc throughout plane: &nbsp;50</p>


The geometry of the aircraft is then parameterized as follows:

<img src="img/canard00.png" alt="Pic here" style="width:700px">


Hence, the weight of fuselage ($M_f$), wing ($M_w$), canard ($M_c$), and vertical tail ($M_t$) will be calculated as 

\begin{align}
        \bullet \quad & M_f = 4\rho_c w l
    \\
        \bullet \quad & M_w = 2 \rho_f \Big[ (\frac{c_{wt}+c_{wr}}{2}) \frac{b_w/2}{\cos\Lambda_w} (\frac{c_{wt}+c_{wr}}{2})0.5 t_w^* \Big]
    \\ & \qquad = \rho_f \frac{b_w/2}{\cos\Lambda_w} (\frac{c_{wt}+c_{wr}}{2})^2t_w^{*}
    \\
        \bullet \quad & M_c = \rho_f \frac{b_c/2}{\cos\Lambda_c} (\frac{c_{ct}+c_{cr}}{2})^2t_c^*
    \\
        \bullet \quad & M_t = \rho_f \frac{l_r+l_t}{2}h t_t
\end{align}

with $\rho_f = 24.8\text{ kg/m}^3$ volumetric density of foam, and $\rho_c = 3.6 \text{ kg/m}^2$ the area density of cardboard. 

In [1]:
using PyPlot

# ------ CONSTANTS
g = 9.82          # (m/s^2) gravity
rhoinf = 1.225    # (kg/m^3) air density at sea level and 15Cdegs
mu = 1.846/10^5   # (kg/m*s) air dynamic viscosity

# ------ MATERIALS
rho_c = 3.6       # (kg/m^2) Cloroplast area density
rho_f = 24.8        # (kg/m^3) HD EPS foam density

# ------ GEOMETRIC PARAMETERS
# Fuselage
w = 8/100         # (m) width
l = 0.75          # (m) length
# Wing
b_w = 0.95        # (m) span
lambda_w = 30*pi/180        # (rad) sweep
c_wtip = 0.10     # (m) tip chord
c_wroot = 0.20    # (m) root chord
t_w = 0.1         # (ratio) ave max thickness/chord
barc_w = (c_wtip+c_wroot)/2
AR_w = b_w/c_wtip # Aspect ratio
tr_w = c_wtip/c_wroot      # Taper ratio
twist_wtip = 0*pi/180      # Tip twist
twist_wroot = 4.0*pi/180   # Root twist
gamma_w = 0*pi/180# Dihedral
# canard
b_c = 0.5*b_w     # (m) span
lambda_c = 0*pi/180        # (rad) sweep
c_ctip = 0.10     # (m) tip chord
c_croot = 0.10    # (m) root chord
t_c = 0.15        # (ratio) ave max thickness/chord
barc_c = (c_ctip+c_croot)/2
AR_c = b_c/c_ctip   # Aspect ratio
tr_c = c_ctip/c_croot      # Taper ratio
twist_ctip = 5*pi/180      # Tip twist
twist_croot = 5*pi/180   # Root twist
gamma_c = 0*pi/180# Dihedral
# vertical tail
h_t = 0.17        # (m) height
l_troot = 0.20    # (m) root length
l_ttip = l_troot/3# (m) tip length
t_t = 0.04        # (m) thickness
barc_t = (l_ttip+l_troot)/2

Sref = 2*b_w*barc_w      # Reference area

# ------ AIRPLANE ASSEMBLY
x_w, y_w, z_w = l*2/4, 0, w/4     # Wing's position in fuselage
x_c, y_c, z_c = l*1/16, 0, 0.0     # Canard's position in fuselage
x_t, y_t, z_t = l-l_troot, 0, 0.0  # Tail's position in fuselage

# ------ USEFUL FUNCTIONS
calc_Re(Vinf, l) = rhoinf*Vinf*l/mu
calc_qinf(Vinf) = 1/2*rhoinf*Vinf.^2
Vinfmin, Vinfmax = 3, 20 # (m/s)


# ------ ASSUMPTIONS
CLratio = 0.1    # Canard-wing distribution of lift
                 #   CLratio = CLcanard/CLtot

# Mass of electronic components
M_base = (9 + 16 + 47 + 14 + 36 + 3 + 32 + 15 + 9*4 + 50)/1000
# Mass of structural components
M_f = rho_c * 4*w*l
M_w = rho_f * b_w/2/cos(lambda_w) * barc_w^2 * t_w
M_c = rho_f * b_c/2/cos(lambda_c) * barc_c^2 * t_c
M_t = rho_f * barc_t * h_t*t_t
# Total mass
Mtot = sum([M_base, M_f, M_w, M_c, M_t])
# Required lift in cruise
L = Mtot*g
println("Total mass: $Mtot (kg)")
println("Lift required: $L (N)")



Total mass: 1.1839256711030754 (kg)
Lift required: 11.626150090232201 (N)


## Drag calculation

The drag will be broken down into parasitic drag ($D_p$) and lifting drag ($D_i$) with components as follow:

\begin{align}
    \bullet \quad & D_p = (D_p)_f + (D_p)_w + (D_p)_c + (D_p)_t
    \\
    \bullet \quad & D_i = (D_i)_w + (D_i)_c + (D_i)_t
\end{align}

with subindices $f,w,c,t$ indicating fuselage, wing, canard, and tail, respectively.



### Parasitic drag

For a flat plate:
<img src="img/friction00.png" alt="Pic here" style="width:400px">

For an airfoil, take the flate plate $(C_{f})_{airfoil} = k(C_{f})_{flatplate}$, with $k$ the form factor

<img src="img/friction01.png" alt="Pic here" style="width:300px">

Is $\Lambda$ the sweep of the wing or the angle of attack at this section? It is sweep. Look at this graph from Prof. Alonso's notes (`etc/JJAlonso_drag.pdf`):

<img src="img/friction02.png" alt="Pic here" style="width:300px">

For a body-of-revolution fuselage we have the following form factor:

<img src="img/friction03.png" alt="Pic here" style="width:300px">

which can be used to calculate a boxed fuselage:

<img src="img/friction04.png" alt="Pic here" style="width:300px">

Verifying the Cf function with a discontinuous step between laminar and turbulent:

In [ ]:
# Coefficient of friction, Cf = tau / qinf
Cf_lam(Re) = 1.328/sqrt(Re)              # Average coeff laminar flow over flat plate
Cf_tur(Re) = 0.455/(log10(Re))^(2.58)    # Average coeff turbulent flow over flat plate
Cf(Re) = Re<5*10^5 ? Cf_lam(Re) : Cf_tur(Re)
# Form factors
k_lift(t_over_c, sweep) = 1 + 2*cos(sweep)*t_over_c + 100*(t_over_c)^4        # Lifting surface
k_revbody(l_over_d) = l_over_d<15 ? 1.675-0.09*l_over_d+0.003*l_over_d^2 : 1  # Body of revolution

fig = figure("Cf_Re", figsize=(7*2,5*1))

subplot(121)
x = [xi for xi in 1000:10^6/100:10^6]
y = [Cf(xi) for xi in x]
plot(x,y, "k""")
grid(true, color="0.8", linestyle="--")
xlabel("Reynolds number")
ylabel(L"C_f")
title("Coefficient of friction")

subplot(122)
x = [Vinf for Vinf in Vinfmin:0.5:Vinfmax]
for (label, lref) in [("Wing", barc_w), ("Canard", barc_c)]
    y = [calc_Re(Vinf, lref) for Vinf in x]
    plot(x,y, label=label)
end
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel("Reynolds Number")
title("Operating Re range")
legend(loc="best")

From the plot on the right we see that we are flying way below the turbulent limit ($<10^5$), so let's assume laminar flow in our calculations.

From $C_f$ of each component, the parasitic drag is calculated as:

<img src="img/friction06.png" alt="Pic here" style="width:300px">
<img src="img/friction05.png" alt="Pic here" style="width:300px">

In [ ]:
function calc_CDp(Vinf; Cftype="transition")
    
    if Cftype=="laminar"
        _Cf = Cf_lam
    elseif Cftype=="turbulent"
        _Cf = Cf_tur
    else
        _Cf = Cf
    end
    
    qinf = calc_qinf(Vinf)
    
    # Fuselage
    deff = sqrt(4*w^2/pi)
    CDp_f = k_revbody(l/deff) * _Cf(calc_Re(Vinf, l)) * (4*w*l)/Sref
    # Wing
    Sexp_w = 2*( (b_w/2 - w/2 )*barc_w )
    CDp_w = k_lift(t_w, lambda_w) * _Cf(calc_Re(Vinf, barc_w)) * (
                            2*(1+0.2*t_w)*Sexp_w)/Sref
    # Canard
    Sexp_c = 2*( (b_c/2 - w/2 )*barc_c )
    CDp_c = k_lift(t_c, lambda_c) * _Cf(calc_Re(Vinf, barc_c)) * (
                            2*(1+0.2*t_c)*Sexp_c)/Sref
    # Horizontal tail
    tail_t_over_c = t_t/barc_t
    lambda_tail = atan2(l_troot-l_ttip, h_t)
    Sexp_t = h_t*barc_t
    CDp_t = k_lift(tail_t_over_c, lambda_tail) * _Cf(
                            calc_Re(Vinf, barc_t)) * (
                            2*(1+0.2*tail_t_over_c)*Sexp_t)/Sref
    
    return [CDp_f, CDp_w, CDp_c, CDp_t]
end



### Induced drag


<img src="img/induced02.png" alt="Pic here" style="width:500px">
<img src="img/induced00.png" alt="Pic here" style="width:300px">
<img src="img/induced01.png" alt="Pic here" style="width:300px">

Hence, after calculating $C_{Dp}$ as explained above, the induced drag will be calculated as a function of $C_L$ as:

\begin{align}
    C_{Di} = \frac{C_L^2}{\pi AR e_{inv}} + KC_{Dp}C_L^2
\end{align}

(no need for calculating the Oswald efficiency factor $e$)

### Total drag

The total drag of each component will then be calculated as 

$$ 
C_{Dtot} = C_{Di} + C_{Dp}
$$

In [ ]:
function calc_CDtot(Vinf; Cftype="transition")
    qinf = calc_qinf(Vinf)
    K = 0.38
    
    # Distributing the lift between lifting bodies
    CL = L / (qinf*Sref)         # Total CL (full airplane)
    CL_t = 0                     # CL on tail
    CL_c = CL*CLratio            # CL on canard
    CL_w = CL - (CL_t+CL_c)      # CL on wing
    
    # Parasitic drag
    CDp = calc_CDp(Vinf; Cftype=Cftype)
    CDp_f, CDp_w, CDp_c, CDp_t = CDp     # Each component's
    
    # Induced drag
    ## Wing
    einv = 0.99*(1-2*(w/b_w)^2)
    AR = b_w / barc_w
    CDiv_w = K*CDp_w*CL_w^2
    CDi_w = CL_w^2/(pi*AR*einv) + CDiv_w
    ## Canard
    einv = 0.99*(1-2*(w/b_c)^2)
    AR = b_c / barc_c
    CDiv_c = K*CDp_c*CL_c^2
    CDi_c = CL_c^2/(pi*AR*einv) + CDiv_c
    ## Vertical tail
    einv = 0.99
    AR = h_t / barc_t
    CDiv_t = K*CDp_t*CL_t^2
    CDi_t = CL_t^2/(pi*AR*einv) + CDiv_t
    
    return [0.0, CDi_w, CDi_c, CDi_t] + CDp
end


And this is what the drag looks like:

In [ ]:
using PyPlot

fig = figure("CDs", figsize=(7*2,5*1))
x = [Vinf for Vinf in Vinfmin:0.5:Vinfmax]
labels = ["Fuselage", "Wing", "Canard", "Tail"]

subplot(121)
ys = [[], [], [], []]
for (i,Vinf) in enumerate(x)
    CDps = calc_CDp(Vinf; Cftype="transition")
    for (j,CDp) in enumerate(CDps)
        push!(ys[j], CDp)
    end
end
for (i,label) in enumerate(labels)
    plot(x,ys[i], label=label)
end
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel(L"C_{Dp}")
title("Parasitic drag coeff per component")
legend(loc="best")

subplot(122)
ys = [[], [], [], []]
for (i,Vinf) in enumerate(x)
    CDs = calc_CDtot(Vinf; Cftype="transition")
    for (j,CD) in enumerate(CDs)
        push!(ys[j], CD)
    end
end
for (i,label) in enumerate(labels)
    plot(x,ys[i], label=label)
end
y = [sum([ys[j][i] for j in 1:4]) for i in 1:size(x)[1]]
plot(x,y, "k", label="Full airplane")
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel(L"C_{Dtot}")
title("Total drag coeff per component")
legend(loc="best")


fig = figure("D", figsize=(7*1,5*1))
for (i,label) in enumerate(labels)
    plot(x,ys[i].*calc_qinf(x)*Sref, label=label)
end
plot(x,y.*calc_qinf(x)*Sref, "k", label="Full airplane")
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel(L"$D_{tot} $  (N)")
title("Total drag per component")
legend(loc="best")


## Performance Analysis

Assuming a jump between laminar and turbulent at $Re=10^5$, lets plot $L/D$ vs $V\infty$:

In [ ]:
ys = [[], [], [], []]
for (i,Vinf) in enumerate(x)
    CDs = calc_CDtot(Vinf; Cftype="transition")
    for (j,CD) in enumerate(CDs)
        push!(ys[j], CD)
    end
end

x = [Vinf for Vinf in Vinfmin:0.5:Vinfmax]
y = [sum([ys[j][i] for j in 1:4]) for i in 1:size(x)[1]]
plot(x,L./(y.*calc_qinf(x)*Sref), "k", label="Full airplane")
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel(L"L/D")
title("Performance vs flight speed")
legend(loc="best")

And here it is assuming turbulent flow all the way:

In [ ]:
ys = [[], [], [], []]
for (i,Vinf) in enumerate(x)
    CDs = calc_CDtot(Vinf; Cftype="turbulent")
    for (j,CD) in enumerate(CDs)
        push!(ys[j], CD)
    end
end

x = [Vinf for Vinf in Vinfmin:0.5:Vinfmax]
y = [sum([ys[j][i] for j in 1:4]) for i in 1:size(x)[1]]
plot(x,L./(y.*calc_qinf(x)*Sref), "k", label="Full airplane")
grid(true, color="0.8", linestyle="--")
xlabel(L"V_\infty \quad (m/s)")
ylabel(L"L/D")
title("Performance vs flight speed")
legend(loc="best")

Hence, the optimum flight speed for this configuration is about 11.0 m/s, with the following performance at that such speed:

In [ ]:
Vinfopt = 11.0 
CLopt = L/(calc_qinf(Vinfopt)*Sref)

println("Optimum flight speed: \t\t$Vinfopt (m/s)")
println("Reynolds number for main wing: \t$(Int64(round(calc_Re(Vinfopt, barc_w),0)))")
println("Lift coefficient (CL): \t\t$(round(CLopt,2))")
println("Max L/D (laminar-turbulent): \t$(round(CLopt/sum(calc_CDtot(Vinfopt; Cftype="transition")),1))")
println("Max L/D (fully turbulent): \t$(round(CLopt/sum(calc_CDtot(Vinfopt; Cftype="turbulent")),1))")

Such values are for the full airplane, so let's split it up between components:

In [ ]:
println("WING")
CL_wing = CLopt*(1-CLratio)
CDtran_wing = calc_CDtot(Vinfopt; Cftype="transition")[2]
CDturb_wing = calc_CDtot(Vinfopt; Cftype="turbulent")[2]
println("\tLift coefficient (CL): \t\t$(round(CL_wing,2))")
println("\tMax L/D (laminar-turbulent): \t$(
                round(CL_wing/CDtran_wing,1))")
println("\tMax L/D (fully turbulent): \t$(
                round(CL_wing/CDturb_wing,1))")

println("CANARD")
CL_can = CLopt*CLratio
CDtran_can = calc_CDtot(Vinfopt; Cftype="transition")[3]
CDturb_can = calc_CDtot(Vinfopt; Cftype="turbulent")[3]
println("\tLift coefficient (CL): \t\t$(round(CL_can,2))")
println("\tMax L/D (laminar-turbulent): \t$(
                round(CL_can/CDtran_can,1))")
println("\tMax L/D (fully turbulent): \t$(
                round(CL_can/CDturb_can,1))")

## Good looking visuals

(for this section Paraview is needed: `>sudo apt-get install paraview` )

The geometry defined above can be visualized running the following command:


In [ ]:
# VLM
include("../code/FLOWVLM20170929/src/FLOWVLM.jl")
vlm = FLOWVLM

save_path = "temp00/"
run_name = "unstallable"

run(`mkdir $save_path`)

# Number of lattices
n = 1
n_w = 20*n
n_c = 10*n
n_t = 5*n
n_f = 2*n
# Lattices expansions
r_w = 5.0
r_c = 5.0
r_t = 1.0
r_f = 1.0


# Creates components
wing = vlm.simpleWing(b_w, AR_w, tr_w, twist_wroot*180/pi,
                  lambda_w*180/pi, gamma_w*180/pi;
                  twist_tip=twist_wtip*180/pi, n=n_w, r=r_w)
canard = vlm.simpleWing(b_c, AR_c, tr_c, twist_croot*180/pi,
                  lambda_c*180/pi, gamma_c*180/pi;
                  twist_tip=twist_ctip*180/pi, n=n_c, r=r_c)

fuselage = vlm.Wing(0.0, -w/2, w/2, l, 0.0)
vlm.addchord(fuselage, 0.0, w/2, w/2, l, 0.0, n_f; r=r_f)
vlm.addchord(fuselage, 0.0, w/2, -w/2, l, 0.0, n_f; r=r_f)
vlm.addchord(fuselage, 0.0, -w/2, -w/2, l, 0.0, n_f; r=r_f)
vlm.addchord(fuselage, 0.0, -w/2, w/2, l, 0.0, n_f; r=r_f)

tail = vlm.Wing(0.0, 0, 0, l_troot, 0.0)
vlm.addchord(tail, l_troot-l_ttip, h_t, 0.0, l_ttip, 0.0, n_t; r=r_t)

# Assembly
system = vlm.WingSystem()
vlm.addwing(system, "Fuselage", fuselage)
vlm.addwing(system, "Wing", wing)
vlm.addwing(system, "Canard", canard)
vlm.addwing(system, "VTail", tail)
vlm.setcoordsystem(system, [x_w,y_w,z_w], [1.0 0 0; 0 1 0; 0 0 1] ;wings=["Wing"])
vlm.setcoordsystem(system, [x_c,y_c,z_c], [1.0 0 0; 0 1 0; 0 0 1] ;wings=["Canard"])
vlm.setcoordsystem(system, [x_t,y_t,z_t], [1.0 0 0; 0 0 1; 0 -1 0] ;wings=["VTail"])

Uinf(X,t) = [1.0, 0, 0]
vlm.setVinf(system, Uinf)
vlm.save(system, run_name; save_horseshoes=true, path=save_path)

strng = save_path
for aux1 in ["Fuselage", "Wing", "VTail", "Canard"]
  strng = strng * run_name * "_"*aux1*"_vlm.vtk;"
end
run(`paraview --data=$(strng)`)
run(`rm -rf $save_path`)


It will pop up paraview displaying something like this:

<img src="img/unstallable01.png" alt="Pic here" style="width:700px">

## Etc

If useful at all, here is the analytical calculation:

<img src="img/IMG_2980.JPG" alt="Pic here" style="width:400px">

In [ ]:
Vinfopt = 12.5 
AR = b_w / barc_w 
_, CDp_w, _, _ = calc_CDp(Vinfopt)
e = 1 / (1/0.98 + 0.38*CDp_w*pi*AR)
CLstar = sqrt(pi*AR*e*CDp_w) 
CLnum = L/(calc_qinf(Vinfopt)*Sref)*(1-CLratio)

L_over_D_star = 1/2*sqrt(pi*AR*e/CDp_w)

println("Analytical optimum CL_{wing}: \t$CLstar")
println("Numerical optimum CL_{wing}: \t$(CLnum)")
println("")
println("Analytical max (L/D)_{wing}: \t$(round(L_over_D_star,1))")
println("Numerical max (L/D)_{wing}: \t$(
            round(CLnum/calc_CDtot(Vinfopt; Cftype="turbulent")[2],1))")

From there we notice that the analytical optimum CL differs from the numerical in about 100%, however, we must remember that the numerical calculation were carried on assuming that not all the lift is generated by the wing, it is rather distributed between the wing and the canard. 

# After Kevin's optimization:

moore54 [9:14 AM] 
Flying at 11 m/s,  4.72145 degrees root, 6.06702 degrees mid 5.36544 degrees tip 4.24697 degrees on the canard, minimum system induced drag for the system weight, einv of the wing is 0.85, and of the system is 0.84.  Any feedback, it takes about 15 min to get a new optimum if we change a something.


moore54 [9:23 AM] 
CL of the system is about 0.3.  Local cl max on the wing is about 0.15.  If we say that the max local cl before stall is 1.4, then the system CL is 1.3 at 20 degrees AOA.  The VLM uses "symmetric" airfoils, i.e. since the NACA 4412 has a 0 lift AOA at -5degrees, we need to subtract 5 degrees off of the twist distribution.


[9:25] 
L/D at the cruise Cl is 16.35

moore54 [9:29 AM] 
At the max AOA of 20 degrees, i.e. the CL max, we can fly at a minimum at 5.4 m/s before we don't produce enough lift to overcome weight.  Stall speed is 5.4m/s


<img src="img/liftdistribution00.png" alt="Pic here" style="width:800px">
<img src="vid/liftdistribution00.gif" alt="Pic here" style="width:800px">
<img src="vid/vortices00.gif" alt="Pic here" style="width:800px">